In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from arcgis.geometry import Geometry, SpatialReference, Point, Polygon
from getpass import getpass
from arcgis.features import FeatureLayer, Feature, FeatureLayerCollection
import pandas as pd
from datetime import datetime

In [ ]:
# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
username = " "
gis = GIS("https://www.arcgis.com", username, getpass())

In [ ]:
find_storymaps = gis.content.search("title: Community Storymap", item_type="Web Mapping Application", max_items=2, outside_org=False)

In [ ]:
#Set up a log file to write errors to
#today = [dt.date.today()]
#log_path = r"T:\CDO_To_Esri\Storymaps\_Archive\Logfiles\Update General Services"
#log_file = os.path.join(log_path, "Update General Services_" + str(today[0]) + str(".txt"))
#out = open(log_file,'w')

for f in find_storymaps:
    try:
        if f.title != 'Alaska DCRA Community Storymaps' and not f.title.startswith('Communit') and 'Community' in str(f.title):
            csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
            sections = csmap.properties["values"]["story"]["sections"]
            community = f.title.replace(" Community Storymap", "").replace(" (Unincorporated Community)", "").replace("'","''")
            print(f.title, f.id)
            if community.startswith("City & Borough of "):
                community = community.replace("City & Borough of ", "")
            if community.startswith("Municipality of "):
                community = community.replace("Municipality of ","")


            #Define the panel and change the main content
            panel = sections[3]
            
        
            region_lyr = gis.content.get('d748e3b58c654b64825f974c25b3c697')
            q = region_lyr.layers[0].query(where="CommunityName = '" + str(community) + "'")
            
            c_ft = q.features[0]
            c_pt = Point(c_ft.geometry)
            print(c_pt)

            base_ext = c_pt.buffer(8000).extent
            
            c_ext = Geometry({
                'xmin':base_ext[0],
                'ymin':base_ext[1],
                'xmax':base_ext[2],
                'ymax':base_ext[3],
                'spatialReference':{
                    'wkid':102006
                }
            }).project_as(3857)
            
            update_wm = WebMap(webmapitem=gis.content.get("8fa71e97183943e2834f00501d7d3e2c"))
            
            lyr_lst = []
            for lyr in update_wm.layers:
                lyr_lst.append(
                {
                    "id": lyr.id,
                    "visibility":True
                })
            
            panel["media"] ={}
            panel["media"] = {
                        "type": "webmap",
                        "webmap": {
                            "id": "8fa71e97183943e2834f00501d7d3e2c",
                            "extent": {
                                "xmin": c_ext.xmin,
                                "ymin": c_ext.ymin,
                                "xmax": c_ext.xmax,
                                "ymax": c_ext.ymax,
                                "spatialReference": {
                                    "wkid": 102100
                                }
                            },
                            "layers": lyr_lst,
                            "popup": None,
                            "overview": {
                                "enable": False,
                                "openByDefault": False
                            },
                            "legend": {
                                "enable": True,
                                "openByDefault": True
                            },
                            "geocoder": {
                                "enable": False
                            },
                            "altText": "Person icon at community location with a pop-up that describes the current population number provided, and a graph showing population from 2011 to present."
                        }
                    }
            
            print('New web map added to panel content.')
            
 



            csmap.save()
            print("Panel webmap updated!!")
    except Exception as e:
        print("-------ISSUE WITH" + str(f.title) + "--------" + str(e))
        #out.write('Unable to update general services map for ' + str(f.title) + ". " + str(e))
        #out.write('\n\n')
        continue